# Batch Extract and align data from Onix, Harp, Sleap, and photometry
## Under development, currently overwrites all the data, will need processing and saving steps at the end of every iteration. But this means iterating accross animals for both photometry and onix data (currently separate)

In [ ]:
import numpy as np
from pathlib import Path
import os
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import pandas as pd
import harp
from pprint import pprint

from harp_resources import process, utils
from sleap import load_and_process as lp

In [ ]:
# Define root directory
root_dir = Path("/Users/rancze/Documents/Data/vestVR/Cohort1/VestibularMismatch_day1/")
data_paths = {}
photometry_paths = {}
# List directories one level down and populate both dictionaries
for sub_dir in root_dir.iterdir():  # Only iterate one level down
    if sub_dir.is_dir() and not sub_dir.name.endswith("_processedData") and not sub_dir.name.startswith("."):
        # Add to data_paths
        data_paths[sub_dir.name] = sub_dir
    if sub_dir.is_dir() and sub_dir.name.endswith("_processedData")and not sub_dir.name.startswith("."):
        processed_photometry_path = sub_dir / "photometry"
        photometry_paths[sub_dir.name] = processed_photometry_path
# Print all the paths
print ("Data paths detected")
for path in data_paths:
    print(path)

In [ ]:
#h1 and h2 only needed if timestamps are readed separately and not as all harp_streams
h1_reader = harp.create_reader('harp_resources/h1-device.yml', epoch=harp.REFERENCE_EPOCH)
h2_reader = harp.create_reader('harp_resources/h2-device.yml', epoch=harp.REFERENCE_EPOCH)
session_settings_reader = utils.SessionData("SessionSettings")
experiment_events_reader = utils.TimestampedCsvReader("ExperimentEvents", columns=["Event"])
onix_framecount_reader = utils.TimestampedCsvReader("OnixAnalogFrameCount", columns=["Index"])
photometry_reader = utils.PhotometryReader("Processed_fluorescence")
video_reader1 = utils.Video("VideoData1")
video_reader2 = utils.Video("VideoData2")
onix_digital_reader = utils.OnixDigitalReader("OnixDigital", columns=["Value.Clock", "Value.HubClock", 
                                                                         "Value.DigitalInputs",
                                                                         "Seconds"])
onix_harp_reader = utils.TimestampedCsvReader("OnixHarp", columns=["Clock", "HubClock", "HarpTime"])

In [ ]:
for key, photometry_path in photometry_paths.items():
    print (" ")
    print(f"Loading Fluoresce: {key}")
    if not cohort2:
        print ("Loading processed fluorescence")
        photometry_data=pd.read_csv(str(photometry_path)+'/Processed_fluorescence.csv')
        print ("Loading processed fluorescence info")
        photometry_info=pd.read_csv(str(photometry_path)+'/Info.csv')
        print ("Loading processed fluorescence events")
        photometry_events=pd.read_csv(str(photometry_path)+'/Events.csv')
    
# Iterate over the dictionary and run the command
for key, data_path in data_paths.items():
    print (" ")
    print(f"Loading: {key}")
    print ("Loading session settings")
    session_settings = utils.load_2(session_settings_reader, data_path)
    print ("Loading experiment events")
    experiment_events = utils.load_2(experiment_events_reader, data_path)
    print ("Loading video data 1")
    video_data1 = utils.load_2(video_reader1, data_path)
    print ("Loading video data 2")
    video_data2 = utils.load_2(video_reader2, data_path)
    print ("Loading OnixDigital")
    onix_digital = utils.load_2(onix_digital_reader, data_path)
    if not cohort0:
        print ("Loading OnixHarp")
        onix_harp = utils.load_2(onix_harp_reader, data_path)
        # removing possible outliers 
        onix_harp = utils.detect_and_remove_outliers(
        df=onix_harp,
        x_column="HarpTime",
        y_column="Clock",
        verbose=False  # True prints all outliers
        )
    print ("Loading OnixAnalogFrameClock")
    onix_analog_framecount = utils.load_2(onix_framecount_reader, data_path)
    print ("Loading OnixAnalogClock")
    onix_analog_clock = utils.read_OnixAnalogClock(data_path)
    print ("Loading OnixAnalogData")
    onix_analog_data = utils.read_OnixAnalogData(data_path, channels = [0], binarise=True) #channels is a list of AI lines, 0-11
    print ("Loading H1 and H2 streams as dict or df (default)")
    harp_streams = utils.load_registers(data_path, dataframe = True) #loads as df, or if False, as dict
print ("Done Loading")

In [ ]:
%whos